In [266]:
import pandas as pd
import numpy as np

ФИЛЬТРАЦИЯ DF ПО ТОВАРАМ-ИНДИКАТОРАМ В СООТВЕТСТВИИ С НАИБОЛЕЕ ПОПУЛЯРНЫМИ РАЗМЕРАМИ ПРЕДМЕТОВ

In [267]:
# читаю файл для получения списка ссылок
urls = pd.read_csv('links.csv')
urls = urls['Ссылка'].tolist()

# из ecom-файла достаю все данные и сохраняю их в DataFrame
df1 = pd.read_csv('ecom.csv')
df1 = df1.drop_duplicates()

# вытаскиваю только наши ссылки для дальнейшего мониторинга
links = []
for url in urls:
    try:
        if '585zolotoy' in url:
            links.append(url)
    except:
        continue

# извлекаю из ссылок артикулы
urls = []
for link in links:
    link = link.replace('https://www.585zolotoy.ru/catalog/products/', '')
    if '/' in link:
        link = link.replace('/', '')
    urls.append(int(link))

# немного фильтрации по отслеживаемым позициям
b = df1[(df1['article'] == 1001247)]
df = df1.query("article.isin(@urls)")
rings = df[((df['type1'] == 'КОЛЬЦО ОБРУЧАЛЬНОЕ') & (df['size'] == 17.5)) | ((df['type1'] == 'КОЛЬЦО') & (df['size'] == 17))].drop_duplicates(['article'])
chains = df[(df['type1'] == 'ЦЕПЬ') & (df['size'] == 45)].drop_duplicates(['article'])
bracelets = df[(df['type1'] == 'БРАСЛЕТ') & (df['size'] == 18)].drop_duplicates(['article'])
earrings = df[(df['type1'] == 'СЕРЬГИ')].drop_duplicates(['article'])
earrings_kongo = df[((df['type1'] == 'СЕРЬГИ-КОНГО') & (df['size'] == 1.5))]
other = df['type1'][(df['type1'] != 'КОЛЬЦО ОБРУЧАЛЬНОЕ') &
                    (df['type1'] != 'КОЛЬЦО') &
                    (df['type1'] != 'ЦЕПЬ') &
                    (df['type1'] != 'БРАСЛЕТ') &
                    (df['type1'] != 'СЕРЬГИ') &
                    (df['type1'] != 'СЕРЬГИ-КОНГО')
                    ].unique().tolist()
df = df.query("type1.isin(@other)")
df = df.drop_duplicates(['article'])

# формирую итоговый DataFrame с товарами-индикаторами 
res = pd.concat([df, rings, chains, bracelets, earrings, earrings_kongo, b], axis=0)
res = res[['article', 'size', 'discount_price', 'label_template']]

In [268]:
# преобразую столбец со скидками в числовой формат
def discount(price):
    try:
        if '-23%' in price:
            return 0.77
        elif '-10%' in price:
            return 0.90
        elif '-15%' in price:
            return 0.85
        return 0
    except:
        return 1
    
res['label_template'] = res['label_template'].apply(discount)